In [0]:
# ---- CONFIG ---- BASE = "dbfs:/FileStore/mini-lakehouse" RAW = f"{BASE}/raw" BRONZE = f"{BASE}/bronze" SILVER = f"{BASE}/silver" GOLD = f"{BASE}/gold"  dbutils.fs.mkdirs(RAW) dbutils.fs.mkdirs(BRONZE) dbutils.fs.mkdirs(SILVER) dbutils.fs.mkdirs(GOLD)

In [0]:

dbutils.notebook.run("./00_setup_data", 60)